In [1]:
import pandas as pd
from openai import OpenAI
import numpy as np
import random
import json

In [2]:
train_solutions = pd.read_excel('data/for_teams/train/solutions.xlsx')
train_tasks = pd.read_excel('data/for_teams/train/tasks.xlsx')
train_tests = pd.read_excel('data/for_teams/train/tests.xlsx')

test_solutions = pd.read_excel('data/for_teams/test/solutions.xlsx')
test_tasks = pd.read_excel('data/for_teams/test/tasks.xlsx')
test_tests = pd.read_excel('data/for_teams/test/tests.xlsx')

In [3]:
def combine_data(solutions_, tasks_, tests_):
    solutions, tasks, tests = solutions_.copy(), tasks_.copy(), tests_.copy()
    
    grouped_tests = tests.groupby('task_id').apply(
        lambda x: x[['number', 'type', 'input', 'output']].to_dict('records')
    ).reset_index()

    grouped_tests = grouped_tests.rename(columns={0: 'tests'})
    
    merged_df = pd.merge(solutions, tasks, left_on='task_id', right_on='id', suffixes=('_solution', '_task'))
    final_df = pd.merge(merged_df, grouped_tests, how='left', on='task_id')
    
    return final_df

In [ ]:
train = combine_data(train_solutions, train_tasks, train_tests)
test = combine_data(test_solutions, test_tasks, test_tests)

/tmp/ipykernel_299594/1392421440.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(
/tmp/ipykernel_299594/1392421440.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(


In [17]:
client = OpenAI(
	base_url="https://api-inference.huggingface.co/v1/",
	api_key="secret"
)


def get_llm_answer(mes, client=client):
    messages = [
    	{ "role": "user", "content": mes }
    ]
    
    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct", 
    	messages=messages, 
    	temperature=1,
    	max_tokens=2048
    )
    
    return completion.choices[0].message.content

In [18]:
get_llm_answer("Привет")

'Привет! Как я могу помочь вам сегодня?'

In [11]:
generated_tasks = pd.read_csv("data/generated_tasks.csv", index_col=0)
generated_tasks

,task,code,inputs,outputs
0,Полина готовится к экзамену по программировани...,"s = list(map(int, input().split()))\nmax_sum =...","['1 2 0 3 4 0 0 5 6', '0 0 7 8 9 0 0 1 0', '1 ...","['7', '24', '3']"
1,"Реализуйте программу, которая проверяет строки...",count = 0\nwhile True:\n line = input()\n ...,"['hello world\npython code\nСТОП', 'one two th...","['1', '0', '1']"
2,"Дан текстовый файл words.txt, содержащий набор...","vowels = 'aeiouAEIOU'\nwith open('words.txt', ...",['Beautiful enchanting nature awaits you withi...,"['enchanting, Beautiful, possibilities', 'adve..."
3,"Реализуйте программу, которая проверяет, может...","def check_access(department, employee_id):\n ...","['research\nR123', 'finance\nF555', 'developme...","['True', 'False', 'True']"
4,"Реализуйте программу, которая проверяет, являю...","words = input().split(', ')\npalindromes = [wo...","['radar, level, cat, civic, dog', 'apple, bana...","['civic, level, radar', 'madam, racecar, rotor..."
...,...,...,...,...
95,"Максим решает, каким блогерам стоит обратиться...","filename = 'bloggers.txt'\nwith open(filename,...",['This is a test file @blogger1 mentioned some...,"['@blogger2', '@blogger5', '@someone']"
96,Василий организует домашнюю библиотеку и хочет...,book_statuses = input().split('\n')\nold_books...,"['Древо жизни, хорошее\\nСтарый дом, обветшало...","['Старый дом', 'Земля и небо, Основание', 'Кни..."
97,"Николай хочет узнать, на какую сумму скидок он...","stores_discounts = input().split(', ')\nmax_di...","['Магазин1: 10%, Магазин2: 15%, Магазин3: 15%'...","['Магазин2, Магазин3', 'БутикА', 'Shop1, Shop3']"
98,"Никита увлекается книгами и решил выяснить, ск...",n = int(input())\nauthors = [input().strip() f...,"['5\nТолстой\nТолстой\nЧехов\nЧехов\nЧехов', '...","['Чехов', 'Толстой', 'Гоголь']"


# Генерируем с моделью попроще неправильный код и небольшие подсказки, где сделана ошибка

In [12]:
promt = """Я тебе дам задачу и код для ее решения. Придумай код с ошибкой в синтаксисе, логике или с опечаткой. Сделай только одну ошибки в коде так, чтобы он
либо давал неправильный ответ, либо не компилировался. Ошибка не должна быть похожа на примеры. Также напиши подсказку, где ты сделал ошибку, подсказка одна из трех видов:
- "ошибка в синтаксисе"
- "ошибка в логике"
- "ошибка с опечаткой"
Ответ я хочу получить в виде json:
{"code": "Неправильный код", "help": "ошибка в ..."}
Не пиши комментариев в коде! Не пиши лишних символов, ковычек! Я жду от тебя только json, который я буду парсить. Заменяй все \n на \\n
Вот небольшой пример:
Запрос:
Задача: Будут введены через строку два целых числа a и b. Выведите их сумму.
Код:
a = int(input())
b = int(input())
print(a + b)
Ответ:
{"code": "a = int(input())
b = int(input())
print(a + b))", "help": "ошибка в синтаксисе"}
Запрос:
"""

In [13]:
false_code = pd.DataFrame({
    "task_id": [],
    "code": [],
    "false_code": [],
    "help": []
})

In [14]:
false_code = pd.read_csv("data/false_code.csv", index_col=0)
false_code

,task_id,code,false_code,help
0,0,"s = list(map(int, input().split()))\nmax_sum =...","s = list(map(int, input().split()))\nmax_sum =...",ошибка в логике
1,0,"s = list(map(int, input().split()))\nmax_sum =...","s = list(map(int, input().split()))\nmax_sum =...",ошибка с опечаткой
2,0,"s = list(map(int, input().split()))\nmax_sum =...","s = list(map(int, input().split()))\nmax_sum =...",ошибка в логике
3,0,"s = list(map(int, input().split()))\nmax_sum =...","s = list(map(int, input().split()))\nmax_sum =...",ошибка с опечаткой
4,0,"s = list(map(int, input().split()))\nmax_sum =...","s = list(map(int, input().split()))\nmax_sum =...",ошибка в логике
...,...,...,...,...
678,50,count = 0\nwhile True:\n user_id = input()....,count = 0\nwhile True:\n user_id = input()....,ошибка в логике
679,50,count = 0\nwhile True:\n user_id = input()....,count = 0\nwhile True:\n user_id = input()....,ошибка в опечатке
680,50,count = 0\nwhile True:\n user_id = input()....,count = 0\nwhile True:\n user_id = input()....,ошибка в синтаксисе
681,51,"lower, upper, number = map(int, input().split(...","lower, upper, number = map(int, input().split(...",ошибка в логике


In [ ]:
for i, row in generated_tasks[51:].iterrows():
    code = row.code
    dscr = row.task
    txt = f"{promt}Задача: {dscr}"
    for j in range(15):
        try:
            txt += f"\nКод:\n{code}\nОтвет:\n"
            ans = get_llm_answer(txt)
            print(ans)
            ans = ans.replace("`", '')
            if ans[:4] == "json": ans = ans[4:]
            data = json.loads(ans)
            resp = {
                "task_id": i,
                "code": code,
                "false_code": data["code"],
                "help": data["help"]
            }
            false_code.loc[len(false_code)] = resp
            print((i*15 + j), resp)
        except:
            pass
    false_code.to_csv("data/false_code.csv")

{"code": "lower, upper, number = map(int, input().split(', '))
if lower <= number < uppper:
    print('Входит в диапазон')
else:
    print('Не входит в диапазон')", "help": "ошибка с опечаткой"}
{"code": "lower, upper, number = map(int, input().split(', '))
if lower <= number < upper:
    print('Входит в диапазон')
elze:
    print('Не входит в диапазон')", "help": "ошибка в синтаксисе"}
{"code": "lower, upper, number = map(int, input().split(', '))
if lower <= number < upper:
    print('Входит в диапазон')
els:
    print('Не входит в диапазон')", "help": "ошибка в синтаксисе"}
{"code": "lower, upper, number = map(int, input().split(', '))
if lower <= number < uper:
    print('Входит в диапазон')
else:
    print('Не входит в диапазон')", "help": "ошибка с опечаткой"}
{"code": "lower, upper, number = map(int, input().split(', '))
if lower <= number < upper
    print('Входит в диапазон')
else:
    print('Не входит в диапазон')", "help": "ошибка в синтаксисе"}
{"code": "lower, upper, numbe